In [1]:
import h3
import folium
import requests
# !pip install git+https://github.com/sabman/h3-utils.git@main#egg=h3_utils
from h3_utils import tools as h3_tools

In [2]:
# get the data from the server (could be a request to geodb-cartodb)
url = "https://gist.githubusercontent.com/sabman/f1aee8de222c263fb0aa4d40409404e7/raw/118150c1d402d5a998a5b180731a98ac33983dad/jettyroad.geojson"
r = requests.get(url)
data = r.json()

In [3]:
# We only need the geometry fro the next steps
geoJson = data['features'][0]['geometry']

In [4]:
polyline = geoJson['coordinates'][0]
# reverse the coordinates to be [lat, lng] since geojson is [x, y] i.e [lng, lat] but folium likes [lat, lng]
polyline = list(map(lambda v: [v[1], v[0]], polyline))
polyline.append(polyline[0])
lat = [p[0] for p in polyline]
lng = [p[1] for p in polyline]
avg_center = [sum(lat)/len(lat), sum(lng)/len(lng)]

# create a map centered on the feature
m = folium.Map(
    location=avg_center,
    zoom_start=13,
    tiles='cartodbpositron'
)

my_PolyLine = folium.PolyLine(
    locations=polyline,
    weight=8, 
    color="green"
)
m.add_child(my_PolyLine)

# ===================== This is important =====================
# find the largest h3 cell who's center falls within the geometry of our feature
# starting our search with resolution 6 (since our athena partitions are at 6)
# once found if its not 6 then find the parent at resolution 6
found_hexagons, level = h3_tools.find_cells_for_geojson(geoJson, 6)
hexagons = found_hexagons
# if level > 6:
#     hexagons = [h3.h3_to_parent(h, 6) for h in found_hexagons]
# else:
#     hexagons = found_hexagons

polylines = []
lat = []
lng = []
for hex in hexagons:
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    # flatten polygons into loops.
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    lat.extend(map(lambda v: v[0], polyline))
    lng.extend(map(lambda v: v[1], polyline))
    polylines.append(polyline)
for polyline in polylines:
    my_PolyLine = folium.PolyLine(locations=polyline, weight=8, color='red')
    m.add_child(my_PolyLine)

display(m)

# How to use `find_resolution_for_geojson`

This function is used to find an h3 resolution that matches the requirements for coverage of the input geojson by the ratio of the area covering the geojson to the total area of the cells. The function will return the lowest resolution that meets the requirements.

See example usage below:

In [5]:
resolution, cells = h3_tools.find_resolution_for_geojson(geoJson, 0.85, 1)

In [20]:
polyline = geoJson['coordinates'][0]
# reverse the coordinates to be [lat, lng] since geojson is [x, y] i.e [lng, lat] but folium likes [lat, lng]
polyline = list(map(lambda v: [v[1], v[0]], polyline))
polyline.append(polyline[0])
lat = [p[0] for p in polyline]
lng = [p[1] for p in polyline]
avg_center = [sum(lat)/len(lat), sum(lng)/len(lng)]

# create a map centered on the feature
m = folium.Map(
    location=avg_center,
    zoom_start=16.4,
    tiles='cartodbpositron'
)

my_PolyLine = folium.PolyLine(
    locations=polyline,
    weight=4, 
    color="green"
)
m.add_child(my_PolyLine)

hexagons = cells

polylines = []
lat = []
lng = []
for hex in hexagons:
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    # flatten polygons into loops.
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    lat.extend(map(lambda v: v[0], polyline))
    lng.extend(map(lambda v: v[1], polyline))
    polylines.append(polyline)
for polyline in polylines:
    my_PolyLine = folium.PolyLine(locations=polyline, weight=4, color='red')
    m.add_child(my_PolyLine)

display(m)